In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.colors
import statsmodels.formula.api as smf
import seaborn as sns
from cem.match import match
from cem.coarsen import coarsen
from cem.imbalance import L1
import statsmodels.api as sm


In [2]:
matplotlib.rc('font', family='Arial') 

In [3]:

pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df=pop_df[['Country Code','2019']].dropna()
pop_df['2019']=pop_df['2019'].astype(int)
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values
possible_countries=[x.lower() for x in possible_countries]

excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS",   # Timor-Leste
    "GEO", #Georgia
    'SWZ'
]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]


possible_iso=list(set(possible_countries)-set(excluded_iso3_codes))

In [11]:
df = pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df=df[df['country'].isin(possible_iso)]
df.rename(columns={'aggregated_value': 'count', 'country': 'Mention_country', 'affiliation_country': 'Aff_country'}, inplace=True)
df = df[df['year'].isin(list(range(2002, 2008))+list(range(2014, 2020)))]
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=['MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
df=df[~df['subjarea'].isin(physical_sciences)]
df['post']=df['year'].apply(lambda x: 0 if x>=2002 and x<=2010 else 1 )

df_sample=df.groupby(['Mention_country','post'])['count'].sum().reset_index()
df_sample['count']/=9
df_sample['log_count']=np.log10(df_sample['count'])
df_pivoted = df_sample.pivot(index=['Mention_country'], columns='post', values='count').reset_index().fillna(0)
df_pivoted.rename(columns={0:'count_pre_ArabSpring', 1:'count_post_ArabSpring'}, inplace=True)
df_pivoted['difference']=(df_pivoted['count_post_ArabSpring']-df_pivoted['count_pre_ArabSpring'])
df_pivoted['rank_pre_ArabSpring']=df_pivoted['count_pre_ArabSpring'].rank(pct=True)
df_pivoted['rank_post_ArabSpring']=df_pivoted['count_post_ArabSpring'].rank(pct=True)
df_pivoted['rank_difference']=df_pivoted['difference'].rank(pct=True)

In [15]:

for c in abbr:
    matched=df_sample['Mention_country'].unique()
    possible_match=list(set(matched)-set(abbr))
    possible_match=possible_match+[c]
    print(possible_match)
    df_analysis=df_sample[df_sample['Mention_country'].isin(possible_match)]
    df_analysis['treated']=df_analysis['Mention_country']==c
    df_analysis['treated']=df_analysis['treated'].astype(int)
    model = smf.quantreg('count ~ treated + post + treated:post', data=df_analysis).fit()
    print(model.summary())

['pan', 'ken', 'tcd', 'tgo', 'som', 'pol', 'gha', 'mys', 'dza', 'dji', 'dnk', 'phl', 'khm', 'bwa', 'hrv', 'rus', 'swe', 'sgp', 'ven', 'nam', 'tjk', 'blr', 'isr', 'mrt', 'arm', 'aut', 'bol', 'aus', 'idn', 'gmb', 'chl', 'ltu', 'qat', 'cyp', 'zmb', 'per', 'mwi', 'lva', 'can', 'rou', 'bgr', 'nzl', 'lao', 'bel', 'irn', 'cze', 'che', 'moz', 'hti', 'afg', 'ita', 'lbr', 'eth', 'lbn', 'aze', 'zwe', 'bfa', 'ago', 'tur', 'col', 'mus', 'alb', 'uzb', 'ury', 'nga', 'slv', 'lso', 'ben', 'pry', 'eri', 'rwa', 'sau', 'svk', 'ecu', 'kgz', 'bih', 'sen', 'vnm', 'are', 'cmr', 'irq', 'jam', 'tza', 'est', 'grc', 'jpn', 'pak', 'uga', 'tha', 'esp', 'ner', 'kaz', 'bdi', 'ukr', 'cri', 'mdg', 'prt', 'nld', 'gbr', 'zaf', 'gab', 'hun', 'arg', 'hnd', 'mli', 'npl', 'dom', 'svn', 'fin', 'kor', 'hkg', 'usa', 'mmr', 'caf', 'bra', 'cub', 'lka', 'deu', 'nic', 'tkm', 'mex', 'prk', 'pse', 'civ', 'nor', 'tto', 'bgd', 'pri', 'mng', 'gtm', 'ind', 'fra', 'sle', 'mkd', 'mda', 'chn', 'egy']
                         QuantReg Regres

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29724\1049729053.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['treated']=df_analysis['Mention_country']==c
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29724\1049729053.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['treated']=df_analysis['treated'].astype(int)
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29724\1049729053.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [16]:
def find_matches(c):
    rank_country=df_pivoted[df_pivoted['Mention_country']==c]['rank_pre_ArabSpring'].values[0]
    return df_pivoted[(df_pivoted['rank_pre_ArabSpring']>=rank_country-0.05) &(df_pivoted['rank_pre_ArabSpring']<=rank_country+0.05)]['Mention_country'].values

In [19]:

for c in abbr:
    matched=find_matches(c)
    possible_match=list(set(matched)-set(abbr))
    possible_match=possible_match+[c]
    print(possible_match)
    df_analysis=df_sample[df_sample['Mention_country'].isin(possible_match)]
    df_analysis['treated']=df_analysis['Mention_country']==c
    df_analysis['treated']=df_analysis['treated'].astype(int)
    model = smf.quantreg('count ~ treated + post + treated:post', data=df_analysis).fit()
    print(model.summary())

['ken', 'bel', 'hun', 'pak', 'vnm', 'cze', 'sgp', 'per', 'aut', 'tza', 'prt', 'sau', 'mys', 'idn', 'egy']
                         QuantReg Regression Results                          
Dep. Variable:                  count   Pseudo R-squared:               0.5032
Model:                       QuantReg   Bandwidth:                       130.1
Method:                 Least Squares   Sparsity:                        298.6
Date:                Wed, 05 Mar 2025   No. Observations:                   30
Time:                        16:25:06   Df Residuals:                       26
                                        Df Model:                            3
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      300.6375     39.904      7.534      0.000     218.614     382.661
treated          1.8849    154.546      0.012      0.990    -315.789     319.559
post           67

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29724\466642402.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['treated']=df_analysis['Mention_country']==c
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29724\466642402.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['treated']=df_analysis['treated'].astype(int)
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29724\466642402.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using